# Traits and Generics

范型和组合天下第一

## Using Traits
There is one unusual rule about trait methods: the trait itself must be in scope. Otherwise, all its methods are hidden:

In [4]:
let mut buf: Vec<u8> = vec![];
buf.write_all(b"hello")?; // error: no method named `write_all`

Error: no method named `write_all` found for struct `Vec<u8>` in the current scope

In [5]:
use std::io::Write;
let mut buf: Vec<u8> = vec![];
buf.write_all(b"hello")?; // ok

The reason `Clone` and `Iterator methods work without any special imports is that they’re always in scope by default: **they’re part of the standard prelude, names that Rust automatically imports into every module**. In fact, the prelude is mostly a carefully chosen selection of traits.

C++ and C# programmers will already have noticed that trait methods are like virtual methods. Still, calls like the one shown above are fast, as fast as any other method call. **Simply put, there’s no polymorphism here**. It’s obvious that `buf` is a vector, not a file or a network connection. The compiler can emit a simple call to `Vec<u8>::write()`. It can even inline the method. (C++ and C# will often do the same, although the possibility of subclassing sometimes precludes this.) Only calls through `&mut dyn Write` incur the overhead of a dynamic dispatch, also known as a virtual method call, which is indicated by the dyn keyword in the type. `dyn Write` is known as a trait object; we’ll look at the technical details of trait objects, and how they compare to generic functions, in the following sections.


### Trait Objects
There are two ways of using traits to write polymorphic code in Rust: trait objects and generics.

Rust doesn’t permit variables of type `dyn Write`:

In [2]:
use std::io::Write;
let mut buf: Vec<u8> = vec![];
let writer: dyn Write = buf; // error: `Write` does not have a constant size

Error: mismatched types

Error: the size for values of type `dyn std::io::Write` cannot be known at compilation time


A variable’s size has to be known at compile time, and types that implement `Write` can be any size.

This may be surprising if you’re coming from C# or Java, but the reason is simple. In Java, a variable of type `OutputStream` (the Java standard interface analogous to `std::io::Write`) is a reference to any object that implements `OutputStream`. The fact that it’s a reference goes without saying. It’s the same with interfaces in C# and most other languages.

In [6]:
use std::io::Write;
{
    let mut buf: Vec<u8> = vec![];
    let writer: &mut dyn Write = &mut buf; // ok
};

A reference to a trait type, like writer, is called a trait object. Like any other reference, a trait object points to some value, it has a lifetime, and it can be either mut or shared.

What makes a trait object different is that Rust usually doesn’t know the type of the referent at compile time. So a trait object includes a little extra information about the referent’s type. This is strictly for Rust’s own use behind the scenes: when you call `writer.write(data)`, Rust needs the type information to dynamically call the right write method depending on the type of `*writer`. You can’t query the type information directly, and Rust does not support downcasting from the trait object `&mut dyn Write` back to a concrete type like `Vec<u8>`.

**Trait object layout**
In memory, a trait object is a fat pointer consisting of a pointer to the value, plus a pointer to a table representing that value’s type. Each trait object therefore takes up two machine words
![image.png](asset/ch11/1.png)
C++ has this kind of run-time type information as well. It’s called a virtual table, or vtable. In Rust, as in C++, the vtable is generated once, at compile time, and shared by all objects of the same type. Everything shown in the darker shade in Figure, including the vtable, is a private implementation detail of Rust. Again, these aren’t fields and data structures that you can access directly. Instead, the language automatically uses the vtable when you call a method of a trait object, to determine which implementation to call.

Seasoned C++ programmers will notice that Rust and C++ use memory a bit differently. In C++, the vtable pointer, or `vptr`, is stored as part of the struct. Rust uses fat pointers instead. The struct itself contains nothing but its fields. This way, a struct can implement dozens of traits without containing dozens of `vptrs`. Even types like `i32`, which aren’t big enough to accommodate a `vptr`, can implement traits.

Rust automatically converts ordinary references into trait objects when needed. This is why we’re able to pass `&mut local_file` to `say_hello` in this example:
```Rust
let mut local_file = File::create("hello.txt")?; say_hello(&mut local_file)?;
```
The type of `&mut local_file` is `&mut File`, and the type of the argument to `say_hello` is `&mut dyn Write`. Since a `File` is a kind of writer, Rust allows this, automatically converting the plain reference to a trait object.
Likewise, Rust will happily convert a `Box<File>` to a `Box<dyn Write>`, a value that owns a writer in the heap:
```Rust
let w: Box<dyn Write> = Box::new(local_file);
```
`Box<dyn Write>`, like `&mut dyn Write`, is a fat pointer: it contains the address of the writer itself and the address of the vtable. The same goes for other pointer types, like `Rc<dyn Write>`.
This kind of conversion is the only way to create a trait object. What the compiler is actually doing here is very simple. At the point where the conversion happens, Rust knows the referent’s true type (in this case, File), so it just adds the address of the appropriate vtable, turning the regular pointer into a fat pointer.

### Generic Functions and Type Parameters
At the beginning of this chapter, we showed a `say_hello()` function that took a trait object as an argument. Let’s rewrite that function as a generic function:



In [8]:
// from
/// The type of out is &mut dyn Write, meaning “a mutable reference to
/// any value that implements the Write trait.” 
use std::io::Write;
fn say_hello(out: &mut dyn Write) -> std::io::Result<()> { 
    out.write_all(b"hello world\n")?;
    out.flush()
}

// to
/// This is a type parameter. It means that throughout the body of 
/// this function, W stands for some type that imple‐ ments the Write trait.
fn say_hello<W: Write>(out: &mut W) -> std::io::Result<()> { 
    out.write_all(b"hello world\n")?;
    out.flush()
}

When you pass `&mut local_file` to the generic `say_hello()` function, you’re calling `say_hello::<File>()`. Rust generates machine code for this function that calls `File::write_all()` and `File::flush()`. When you pass `&mut bytes`, you’re calling `say_hello::<Vec<u8>>()`. Rust generates separate machine code for this version of the function, calling the corresponding `Vec<u8>` methods. In both cases, Rust infers the type `W` from the type of the argument. This process is known as _monomorphization_, and the compiler handles it all automatically.

You can always spell out the type parameters:
```Rust
say_hello::<File>(&mut local_file)?;
```
This is seldom necessary, because Rust can usually deduce the type parameters by looking at the arguments. Here, the `say_hello` generic function expects a `&mut W` argument, and we’re passing it a `&mut File`, so Rust infers that `W = File`.

If the generic function you’re calling doesn’t have any arguments that provide useful clues, you may have to spell it out:

In [13]:
// calling a generic method collect<C>() that takes no arguments 
let v1 = (0 .. 1000).collect(); // error: can't infer type

Error: Couldn't automatically determine type of variable `v1`.
Please give it an explicit type.

In [14]:
let v2 = (0 .. 1000).collect::<Vec<i32>>(); // ok

Sometimes we need multiple abilities from a type parameter. For example, if we want to print out the top ten most common values in a vector, we’ll need for those values to be printable:
```Rust
use std::fmt::Debug;
fn top_ten<T: Debug>(values: &Vec<T>) { ... }
```
But this isn’t good enough. How are we planning to determine which values are the most common? The usual way is to use the values as keys in a hash table. That means the values need to support the `Hash` and `Eq` operations. The bounds on T must include these as well as Debug. The syntax for this uses the + sign:
```Rust
use std::hash::Hash; use std::fmt::Debug;
fn top_ten<T: Debug + Hash + Eq>(values: &Vec<T>) { ... }
```
It’s also possible for a type parameter to have no bounds at all, but you can’t do much with a value if you haven’t specified any bounds for it. You can move it. You can put it into a box or vector. That’s about it.

As this example shows, the bounds can get to be so long that they are hard on the eyes. Rust provides an alternative syntax using the keyword where:
```Rust
fn run_query<M, R>(data: &DataSet, map: M, reduce: R) -> Results 
    where M: Mapper + Serialize,
          R: Reducer + Serialize
{...}
```
The type parameters M and R are still declared up front, but the bounds are moved to separate lines. This kind of where clause is also allowed on generic structs, enums, type aliases, and methods—anywhere bounds are permitted.

A generic function can have both lifetime parameters and type parameters. Lifetime parameters come first:
```Rust
/// Return a reference to the point in `candidates` that's
/// closest to the `target` point.
fn nearest<'t, 'c, P>(target: &'t P, candidates: &'c [P]) -> &'c P
where P: MeasureDistance {
... 
}
```
Lifetimes never have any impact on machine code. Two calls to `nearest()` using the same type `P`, but different lifetimes, will call the same compiled function. Only differing types cause Rust to compile multiple copies of a generic function.

### Which to Use
The choice of whether to use trait objects or generic code is subtle. Since both features are based on traits, they have a lot in common.

Trait objects are the right choice whenever you need a collection of values of mixed types, all together. It is technically possible to make generic salad:
```Rust
trait Vegetable { 
    ...
}
struct Salad<V: Vegetable> { 
    veggies: Vec<V>
}
```
However, this is a rather severe design. Each such salad consists entirely of a single type of vegetable. Not everyone is cut out for this sort of thing. One of your authors once paid $14 for a `Salad<IcebergLettuce>` and has never quite gotten over the experience.

How can we build a better salad? Since Vegetable values can be all different sizes, we can’t ask Rust for a `Vec<dyn Vegetable>`:

In [17]:
trait Vegetable {}

struct Salad {
    veggies: Vec<dyn Vegetable> // error: `dyn Vegetable` does
                                // not have a constant size
}

Error: the size for values of type `(dyn Vegetable + 'static)` cannot be known at compilation time

In [18]:
trait Vegetable {}

struct Salad {
    veggies: Vec<Box<dyn Vegetable>>
}

Each `Box<dyn Vegetable>` can own any type of vegetable, but the box itself has a constant size—two pointers—suitable for storing in a vector. Apart from the unfortunate mixed metaphor of having boxes in one’s food, this is precisely what’s called for, and it would work out just as well for shapes in a drawing app, monsters in a game, pluggable routing algorithms in a network router, and so on.

使用 generics:
* 没有 dyn 关键字，无需运行时参与，方法和类型在编译期就指定了，于是编译器可以帮助优化代码，速度上快
* 有些特征的 associated function 可能需要范型，单纯的 trait 根本无法指定类型并编译
* 可以绑定多个 trait，但是反过来 rust 不支持 `&mut (dyn Debug + Hash + Eq)`

使用 trait:
* 减少编译的代码量，防止代码膨胀
* 多种类型，更加灵活

更多细节请阅读原书

## Defining and Implementing Traits
Defining a trait is simple. Give it a name and list the type signatures of the trait methods. If we’re writing a game, we might have a trait like this:
```Rust
/// A trait for characters, items, and scenery -
/// anything in the game world that's visible on screen. 
trait Visible {
    /// Render this object on the given canvas.
    fn draw(&self, canvas: &mut Canvas);
    /// Return true if clicking at (x, y) should 
    /// select this object.
    fn hit_test(&self, x: i32, y: i32) -> bool;
}


impl Visible for Broom {
    fn draw(&self, canvas: &mut Canvas) {
        for y in self.y - self.height - 1 .. self.y { 
            canvas.write_at(self.x, y, '|');
        }
        canvas.write_at(self.x, self.y, 'M');
    }
    fn hit_test(&self, x: i32, y: i32) -> bool { 
        self.x == x
        && self.y - self.height - 1 <= y
        && y <= self.y
    }
}
```

Note that this impl contains an implementation for each method of the Visible trait, and nothing else. Everything defined in a trait impl must actually be a feature of the trait; if we wanted to add a helper method in support of `Broom::draw()`, we would have to define it in a separate impl block:
```Rust
impl Broom {
    /// Helper function used by Broom::draw() below. 
    fn broomstick_range(&self) -> Range<i32> {
        self.y - self.height - 1 .. self.y
    }
}
```

These helper functions can be used within the trait impl blocks:
```Rust
impl Visible for Broom {
    fn draw(&self, canvas: &mut Canvas) {
        for y in self.broomstick_range() { 
            ...
        }
        ... 
    }
    ... 
}
```

### Default Methods
The Sink writer type we discussed earlier can be implemented in a few lines of code. First, we define the type:
```Rust
/// A Writer that ignores whatever data you write to it.
pub struct Sink;
```
Sink is an empty struct, since we don’t need to store any data in it. Next, we provide
an implementation of the `Write` trait for `Sink`: 
```Rust
use std::io::{Write, Result};
impl Write for Sink {
    fn write(&mut self, buf: &[u8]) -> Result<usize> {
        // Claim to have successfully written the whole buffer.
        Ok(buf.len())
    }
    fn flush(&mut self) -> Result<()> { 
        Ok(())
    } 
}
```
So far, this is very much like the Visible trait. But we have also seen that the Write trait has a `write_all` method:
```Rust
let mut out = Sink; 
out.write_all(b"hello world\n")?;
```
Why does Rust let us impl `Write` for `Sink` without defining this method? The answer is that the standard library’s definition of the Write trait contains a default implementation for `write_all`:

```Rust
trait Write {
    fn write(&mut self, buf: &[u8]) -> Result<usize>; 
    fn flush(&mut self) -> Result<()>;
    fn write_all(&mut self, buf: &[u8]) -> Result<()> { 
        let mut bytes_written = 0;
        while bytes_written < buf.len() {
            bytes_written += self.write(&buf[bytes_written..])?;
        }
        Ok(()) 
    }
    ... 
}
```

The write and flush methods are the basic methods that every writer must implement. A writer may also implement `write_all`, but if not, the default implementation shown earlier will be used.

The most dramatic use of default methods in the standard library is the `Iterator` trait, which has one required method `(.next())` and dozens of default methods.

### Traits and Other People’s Types
Rust lets you implement any trait on any type, as long as either the trait or the type is introduced in the current crate.
This means that any time you want to add a method to any type, you can use a trait to do it:
```Rust
trait IsEmoji {
    fn is_emoji(&self) -> bool;
}
    
/// Implement IsEmoji for the built-in character type.
impl IsEmoji for char {
    fn is_emoji(&self) -> bool {
        ... 
    }
}
assert_eq!('$'.is_emoji(), false);
```
Like any other trait method, this new `is_emoji` method is only visible when `IsEmoji` is in scope.

The sole purpose of this particular trait is to add a method to an existing type, `char`. This is called an extension trait. Of course, you can add this trait to types, too, by writing impl `IsEmoji` for `str { ... }` and so forth.

You can even use a generic impl block to add an extension trait to a whole family of types at once. This trait could be implemented on any type:
```Rust
/// You can write HTML to any std::io writer.
impl<W: Write> WriteHtml for W {
    fn write_html(&mut self, html: &HtmlDocument) -> io::Result<()> {
        ... 
    }
}
```
The line `impl<W: Write> WriteHtml for W` means “for every type `W` that implements Write, here’s an implementation of `WriteHtml` for `W`.”

The `serde` library offers a nice example of how useful it can be to implement user-defined traits on standard types. `serde` is a serialization library. That is, you can use it to write Rust data structures to disk and reload them later. The library defines a trait, `Serialize`, that’s implemented for every data type the library supports. So in the serde source code, there is code implementing `Serialize` for `bool`, `i8`, `i16`, `i32`, `array` and `tuple` types, and so on, through all the standard data structures like `Vec` and `HashMap`.
The upshot of all this is that `serde` adds a `.serialize()` method to all these types. It can be used like this:
```Rust
use serde::Serialize; // 引入 trait serde::Serialize，赋予了 HashMap<String, String> .serialize() 方法
use serde_json; // 
pub fn save_configuration(config: &HashMap<String, String>) -> std::io::Result<()> {
    // Create a JSON serializer to write the data to a file.
    let writer = File::create(config_filename())?;
    let mut serializer = serde_json::Serializer::new(writer);
    // The serde `.serialize()` method does the rest.
    config.serialize(&mut serializer)?; Ok(())
}
```
We said earlier that when you implement a trait, either the trait or the type must be new in the current crate. This is called the orphan rule. It helps Rust ensure that trait implementations are unique. Your code can’t impl `Write` for `u8`, because both `Write` and `u8` are defined in the standard library. If Rust let crates do that, there could be multiple implementations of `Write` for `u8`, in different crates, and Rust would have no reasonable way to decide which implementation to use for a given method call.

### Self in Traits
A trait can use the keyword `Self` as a type. The standard `Clone` trait, for example, looks like this (slightly simplified):
```Rust
// Clone trait的大致定义方式
pub trait Clone {
    fn clone(&self) -> Self; 
    ...
}
```
Using Self as the return type here means that the type of `x.clone()` is the same as the type of `x`, whatever that might be. If `x` is a String, then the type of `x.clone()` is String—not `dyn` Clone or any other cloneable type.
这个语法设计得真是妙啊。

Likewise, if we define this trait:
```Rust
pub trait Spliceable {
    fn splice(&self, other: &Self) -> Self;
}
```
with two implementations:
```Rust
impl Spliceable for CherryTree {
    fn splice(&self, other: &Self) -> Self {
        ... 
    }
}
impl Spliceable for Mammoth {
    fn splice(&self, other: &Self) -> Self {
        ... 
    }
}
```
then inside the first `impl`, `Self` is simply an alias for `CherryTree`, and in the second, it’s an alias for `Mammoth`.

A trait that uses the `Self` type is incompatible with trait objects:
```Rust
    // error: the trait `Spliceable` cannot be made into an object
    fn splice_anything(left: &dyn Spliceable, right: &dyn Spliceable) { 
        let combo = left.splice(right);
        // ...
    }
```
The reason is something we’ll see again and again as we dig into the advanced features of traits. Rust rejects this code because it has no way to type-check the call `left.splice(right)`. **The whole point of trait objects is that the type isn’t known until run time. Rust has no way to know at compile time if left and right will be the same type, as required.**

Trait objects are really intended for the simplest kinds of traits, the kinds that could be implemented using interfaces in Java or abstract base classes in C++. The more advanced features of traits are useful, but they can’t coexist with trait objects because with trait objects, you lose the type information Rust needs to type-check your program.

Now, had we wanted genetically improbable splicing, we could have designed a trait-object-friendly trait:
```Rust

pub trait MegaSpliceable {
    fn splice(&self, other: &dyn MegaSpliceable) -> Box<dyn MegaSpliceable>;
}
```
This trait is compatible with trait objects. There’s no problem type-checking calls to this `.splice()` method because the type of the argument other is not required to match the type of self, as long as both types are `MegaSpliceable`.

### Subtraits
We can declare that a trait is an extension of another trait:
```Rust
/// Someone in the game world, either the player or some other 
/// pixie, gargoyle, squirrel, ogre, etc.
trait Creature: Visible {
    fn position(&self) -> (i32, i32); fn facing(&self) -> Direction; 
    ...
}
```
The phrase trait Creature: Visible means that all creatures are visible. Every type that implements Creature must also implement the Visible trait:
```Rust
impl Visible for Broom { 
    ...
}
impl Creature for Broom {
    ... 
}
```
We can implement the two traits in either order, but it’s an error to implement `Creature` for a type without also implementing `Visible`. Here, we say that `Creature` is a subtrait of `Visible`, and that `Creature` is `Visible`’s supertrait.

In Rust, a subtrait does not inherit the associated items of its supertrait; each trait still needs to be in scope if you want to call its methods. In fact, Rust’s subtraits are really just a shorthand for a bound on `Self`. A definition of `Creature` like this is exactly equivalent to the one shown earlier:
```Rust
trait Creature where Self: Visible { 
    ...
}
```

### Type-Associated Functions
Like Java and C# interfaces, trait objects don’t support type-associated functions. If you want to use `&dyn StringSet` trait objects, you must change the trait, adding the bound where `Self: Sized` to each associated function that doesn’t take a self argument by reference: (这句话非常非常重要，如果希望一个 trait 作为动态对象，就要将没有把 &self 作为参数的方法全部加上 Self: Sized 的限制，它表明这个函数只能被那些具有确定大小的类型调用，而不能被不确定大小的动态对象调用。)
```Rust
trait StringSet { 
    fn new() -> Self
        where Self: Sized;
    fn from_slice(strings: &[&str]) -> Self
        where Self: Sized;
    fn contains(&self, string: &str) -> bool;
    fn add(&mut self, string: &str); 
}
```
This bound tells Rust that trait objects are excused from supporting this particular associated function. With these additions, `StringSet` trait objects are allowed; they still don’t support `new` or `from_slice`, but you can create them and use them to call `.contains()` and `.add()`. The same trick works for any other method that is incompatible with trait objects.

In [3]:
trait StringSet {
    /// Return a new empty set. 
    fn new() -> Self;
    /// Return a set that contains all the strings in `strings`.
    fn from_slice(strings: &[&str]) -> Self;
    /// Find out if this set contains a particular `value`.
    fn contains(&self, string: &str) -> bool;
    /// Add a string to this set.
    fn add(&mut self, string: &str); 
}

/// Return the set of words in `document` that aren't in `wordlist`.
fn unknown_words<S: StringSet>(document: &[String], wordlist: &S) -> S { 
    let mut unknowns = S::new();
    for word in document {
        if !wordlist.contains(word) { 
            unknowns.add(word);
        } 
    }
    unknowns
}

In [6]:
// 仔细阅读这段报错提示
trait StringSet {
    /// Return a new empty set. 
    fn new() -> Self;
    /// Return a set that contains all the strings in `strings`.
    fn from_slice(strings: &[&str]) -> Self;
    /// Find out if this set contains a particular `value`.
    fn contains(&self, string: &str) -> bool;
    /// Add a string to this set.
    fn add(&mut self, string: &str); 
}

fn foo(obj: &dyn StringSet) -> () {
}

Error: the trait `StringSet` cannot be made into an object

In [8]:
// 这样就好了
trait StringSet {
    /// Return a new empty set. 
    fn new() -> Self
        where Self: Sized;
    /// Return a set that contains all the strings in `strings`.
    fn from_slice(strings: &[&str]) -> Self
        where Self: Sized;
    /// Find out if this set contains a particular `value`.
    fn contains(&self, string: &str) -> bool;
    /// Add a string to this set.
    fn add(&mut self, string: &str); 
}

fn foo(obj: &dyn StringSet) -> () {
}

In [11]:
// 想要把associate func放到trait里面，就算函数签名里没有Self，也要加上
// where Self: Sized; 的约束
trait StringSet {
    /// Return a new empty set. 
    fn new() -> Self
        where Self: Sized;
    /// Return a set that contains all the strings in `strings`.
    fn from_slice(strings: &[&str]) -> Self
        where Self: Sized;
    fn foobar()
        where Self: Sized;
    /// Find out if this set contains a particular `value`.
    fn contains(&self, string: &str) -> bool;
    /// Add a string to this set.
    fn add(&mut self, string: &str); 
}

fn foo(obj: &dyn StringSet) -> () {
}

## Fully Qualified Method Calls


In [4]:
"hello".to_string();
// following three are qualified method calls.
str::to_string("hello");
ToString::to_string("hello");
<str as ToString>::to_string("hello"); 
// the implementation of that method, the trait, the method of that trait and the value to which that implementation is being applied.
// in some cases you need a way to say exactly what you mean. Fully qualified method calls fit the bill.

When you write `"hello".to_string()`, using the `.` operator, you don’t say exactly which `to_string` method you’re calling. Rust has a method lookup algorithm that figures this out, depending on the types, deref coercions, and so on. With fully qualified calls, you can say exactly which method you mean, and that can help in a few odd cases:
* When two methods have the same name. The classic hokey example is the Outlaw with two .draw() methods from two different traits, one for drawing it on the screen and one for interacting with the law:
```Rust
outlaw.draw(); // error: draw on screen or draw pistol? 
Visible::draw(&outlaw); // ok: draw on screen
HasPistol::draw(&outlaw); // ok: corral
```
    Usually you’re better off renaming one of the methods, but sometimes you can’t.
* When the type of the self argument can’t be inferred:
```Rust
let zero = 0; // type unspecified; could be `i8`, `u8`, ...
zero.abs(); // error: can't call method `abs` 
            // on ambiguous numeric type
i64::abs(zero); // ok
```
* When using the function itself as a function value:
```Rust
let words: Vec<String> =
    line.split_whitespace() // iterator produces &str values
        .map(ToString::to_string) // ok 
        .collect();
```
* When calling trait methods in macros. 

## Traits That Define Relationships Between Types
### Associated Types (or How Iterators Work)
Rust has a standard `Iterator` trait, defined like this:
```Rust
pub trait Iterator {
    type Item;
    fn next(&mut self) -> Option<Self::Item>;
    ...
}
```
The first feature of this trait, `type Item`;, is an associated type. Each type that implements `Iterator` must specify what type of item it produces.

The second feature, the `next()` method, uses the associated type in its return value. `next()` returns an `Option<Self::Item>`: either `Some(item)`, the next value in the sequence, or `None` when there are no more values to visit. The type is written as `Self::Item`, not just plain Item, because `Item` is a feature of each type of iterator, not a standalone type. As always, self and the Self type show up explicitly in the code everywhere their fields, methods, and so on are used.
Here’s what it looks like to implement Iterator for a type:
```Rust
// (code from the std::env standard library module)
impl Iterator for Args { type Item = String;
    fn next(&mut self) -> Option<String> { 
        ...
    }
    ... 
}
```
Generic code can use associated types:
```Rust
/// Loop over an iterator, storing the values in a new vector.
fn collect_into_vector<I: Iterator>(iter: I) -> Vec<I::Item> { 
    let mut results = Vec::new();
    for value in iter {
        results.push(value);
    }
    results
}
```
Inside the body of this function, Rust infers the type of value for us, which is nice; but we must spell out the return type of `collect_into_vector`, and the `Item` associated type is the only way to do that. (`Vec<I>` would be simply wrong: we would be claiming to return a vector of iterators!)

In [6]:
/// Print out all the values produced by an iterator
fn dump<I>(iter: I) 
    where I: Iterator
{
    for (index, value) in iter.enumerate() {
        println!("{}: {:?}", index, value); // error
    } 
}

Error: `<I as Iterator>::Item` doesn't implement `Debug`

In [8]:
use std::fmt::Debug;

fn dump<I>(iter: I) 
    where I: Iterator, I::Item: Debug
{
    for (index, value) in iter.enumerate() {
        println!("{}: {:?}", index, value); // error
    } 
}

In [9]:
use std::fmt::Debug;

fn dump<I>(iter: I) 
    where I: Iterator<Item = String>
{
    for (index, value) in iter.enumerate() {
        println!("{}: {:?}", index, value); // error
    } 
}

`Iterator<Item=String>` is itself a trait. If you think of Iterator as the set of all iterator types, then `Iterator<Item=String>` is a subset of Iterator: the set of iterator types that produce Strings. This syntax can be used anywhere the name of a trait can be used, including trait object types:

In [10]:
fn dump(iter: &mut dyn Iterator<Item=String>) { 
    for (index, s) in iter.enumerate() {
        println!("{}: {:?}", index, s);
    }
}

Traits with associated types, like `Iterator`, are compatible with trait methods, but only if all the associated types are spelled out, as shown here. Otherwise, the type of s could be anything, and again, Rust would have no way to type-check this code.

We’ve shown a lot of examples involving iterators. It’s hard not to; they’re by far the most prominent use of associated types. But associated types are generally useful whenever a trait needs to cover more than just methods:
* In a thread pool library, a `Task` trait, representing a unit of work, could have an associated Output type.
* A `Pattern` trait, representing a way of searching a string, could have an associated Match type, representing all the information gathered by matching the pattern to the string:
```Rust
trait Pattern { 
    type Match;
    
    fn search(&self, string: &str) -> Option<Self::Match>; 
}

/// You can search a string for a particular character.
impl Pattern for char {
    /// A "match" is just the location where the 
    /// character was found.
    type Match = usize;
    fn search(&self, string: &str) -> Option<usize> { 
        ...
    } 
}
```
* A library for working with relational databases might have a Database Connection trait with associated types representing transactions, cursors, prepared statements, and so on.

### impl Trait
